# Experimenting with SDOF System using KAN, CNN, and MLP

## 1. SDOF data generation

In [2]:
import numpy as np

def generate_sdoff_data(samples=1000, dt=0.01):
    """Generate synthetic SDOF data."""
    # Parameters for the SDOF system
    m, c, k = 1.0, 1.5, 200.0  # Mass, damping, stiffness
    f = np.random.normal(0, 1, samples)  # External force (white noise)
    
    # Initialize arrays for displacement, velocity, acceleration
    x, v, a = np.zeros(samples), np.zeros(samples), np.zeros(samples)

    # Simulate the response using Newmark-beta method
    for i in range(1, samples):
        a[i] = (f[i] - c * v[i - 1] - k * x[i - 1]) / m
        v[i] = v[i - 1] + a[i] * dt
        x[i] = x[i - 1] + v[i] * dt

    return np.stack([v, a, f], axis=1), x  # Inputs: (velocity, accel, force), Output: displacement

# Generate data
X, y = generate_sdoff_data()
print("Input shape:", X.shape)
print("Output shape:", y.shape)

Input shape: (1000, 3)
Output shape: (1000,)


## 2. Define models

### KAN

In [ ]:
from pykan import KAN

kan = KAN(input_dim=3, output_dim=1, layers=[3, 10, 10, 1])

### CNN

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 100, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

### MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(300, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

## 3. Prepare Data for Training

In [ ]:
import torch

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

# Reshape data for CNN (batch_size, channels, length)
X_cnn = X_tensor.unsqueeze(0).permute(0, 2, 1)  # Example: (1, 3, 100)

## 4. Define Training Loop

In [ ]:
def train_model(model, X, y, epochs=1000, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/1000], Loss: {loss.item():.4f}')

## 5. Train and Compare Models

In [ ]:
# Train KAN
train_model(kan, X_tensor, y_tensor)

# Train CNN
cnn = CNN()
train_model(cnn, X_cnn, y_tensor)

# Train MLP
mlp = MLP()
train_model(mlp, X_tensor.view(-1, 300), y_tensor)


## 6. Evaluate and Visualize Results

In [ ]:
import matplotlib.pyplot as plt

def plot_results(model, X, y_true, title):
    model.eval()
    with torch.no_grad():
        y_pred = model(X).squeeze().numpy()
    
    plt.plot(y_true, label='True Displacement')
    plt.plot(y_pred, label='Predicted Displacement')
    plt.title(title)
    plt.legend()
    plt.show()

# Plot results
plot_results(kan, X_tensor, y, 'KAN Prediction')
plot_results(cnn, X_cnn, y, 'CNN Prediction')
plot_results(mlp, X_tensor.view(-1, 300), y, 'MLP Prediction')
